In [86]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [87]:
def generate_description(diagnosis_category, diagnosis_sub_category, treatment_category, treatment_sub_category, temperature):
    chat = ChatOpenAI(temperature=temperature)
    messages = [
        SystemMessage(
            content=
            """You are programmed to create believable narratives of interactions between patients and doctors within a hospital setting, strictly from the doctor's viewpoint. 
            Your responses should draw from specified variables: Diagnosis Category, Diagnosis Sub Category, Treatment Category, and Treatment Sub Category. 
            Exclude any inputs that are empty or contain NaN.
            Do not invent patient identifiers such as names, ages, or genders. 
            Invent plausible symptoms that align with the given diagnosis and treatment. 
            Your output should be a straightforward text description.
            Omit explanations or justifications.
            """
        ),
        HumanMessage(
            content=f'Diagnosis Category: {diagnosis_category}, Diagnosis Sub Category:{diagnosis_sub_category}, Treatment Category:{treatment_category}, Treatment Sub Category:{treatment_sub_category}. Generate description.'
        ),
    ]
    return chat.invoke(messages)

In [2]:
import pandas as pd

df = pd.read_csv('dataset_1.csv')
df


,Visit ID,Visit Date,Patient ID,Diagnosis Category,Diagnosis Sub Category,Treatment Category,Treatment Sub Category,New Patient,Consulting Physician,Patient Name,Patient Gender,Patient Age,Patient Age Range,Patient Blood Type,Patient Insurance Number,Patient Phone,Patient Address,Patient Occupation,Patient Emergency Contact
0,VQC513203,2017-01-01,PZM508653,Chronic Pain,Vertebral Disc Problem,Pharmacy/Prescription Drugs,Non-FDA Approved Use,True,Dr. Jerry Daniels,Michelle Fisher,Female,43,41-50,O-,G264037622,15786233257,Pärtzeltweg 2\n22301 Neunburg vorm Wald,"Surveyor, quantity","Jennifer Bailey, 015680180768."
1,VVC435406,2017-01-01,PSN036517,Endocrine/ Metabolic,Hormone Deficiency,Pharmacy/Prescription Drugs,Hormones,True,Dr. Michelle Lamb,Brooke Davis,Female,36,31-40,O+,Y133547589,16238255302,Kira-Gorlitz-Allee 8\n67100 Rosenheim,Copy,"Dennis Carlson, 015182104709."
2,VKT437745,2017-01-01,PPD253419,Pediatrics,Delayed Speech,Rehabilitation Services - Outpatient,Speech Therapy,True,Dr. Michelle Lamb,Andrew Graves,Male,5,0-10,O+,F115599209,17550508440,Thiesstr. 3/5\n68745 Bremen,Claims inspector/assessor,"Veronica Harris, 015034485673."
3,VJG208744,2017-01-01,PBN488954,Central Nervous System/ Neuromuscular,NaN,Pharmacy/Prescription Drugs,Non-FDA Approved Use,True,Dr. James Barber,Cody Carpenter,Male,52,51-64,AB+,W720918648,15210588122,Schmidtkeallee 53\n54913 Siegen,Tourism officer,"Michelle Graham, 017438105819."
4,VAF235393,2017-01-01,PJG173047,Cancer,Breast Cancer,Cancer Treatment,Surgery,True,Dr. James Barber,Christopher White,Female,36,31-40,A+,Q198981012,16561784363,Ida-Fliegner-Ring 7/6\n53118 Wolgast,Engineering geologist,"Peter Stout, 015017527431."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11684,VTL998110,2018-12-30,PND058072,Mental,Depression,Electrical/ Thermal/ Radiofreq. Interventions,Transcranial Magnetic Stimulation,False,Dr. Alexandria Gaines,Jean Rios,Male,51,51-64,A-,Private,16805112343,Wolfram-auch Schlauchin-Ring 3/3\n94816 Neuruppin,"Research officer, political party","Emily Garrett, 016593887796."
11685,VBT985120,2018-12-30,PIT750921,Orthopedic/ Musculoskeletal,Shoulder Problem,"Diagnostic Imaging, Screening and Testing",MRI,False,Dr. Shelly Hunt,Jesse Terry,Male,52,51-64,B+,U505643033,17924524385,Miesplatz 31\n22468 Bayreuth,"Social research officer, government","Jeanette Rodriguez, 017833888272."
11686,VLH032189,2018-12-30,PPC895292,Cancer,Liver Cancer,Special Procedure,NaN,False,Dr. Eddie Young,Hailey Terry,Male,62,51-64,B-,Q100759966,15775878159,Sölzerring 05\n21768 Altötting,"Engineer, petroleum","Linda Flores, 016973980632."
11687,VKA771318,2018-12-30,PDB455996,Endocrine/ Metabolic,Thyroid Problems,Acute Medical Services - Outpatient,Emergency Room,False,Dr. Michelle Lamb,Christopher Williams,Female,54,51-64,AB+,Private,16951975501,Fiebigplatz 5\n26886 Fürstenwalde,"Chemist, analytical","James Farrell, 015303912796."


In [80]:
import csv
import os

filename = 'dataset_2.csv'
if not os.path.exists(filename):
    with open(filename, 'w', newline='') as csvfile:
        # Define the fieldnames
        fieldnames = ["Visit ID", "Description"]
        # Create a writer object
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        # Write the header
        writer.writeheader()

In [91]:
filename = 'dataset_2.csv'
def append_row(visit_id, description):
    visit_id = str(visit_id)
    description = str(description)
    row_data = {"Visit ID": visit_id, "Description": description}
    
    with open(filename, 'a', newline='') as csvfile:
        fieldnames = row_data.keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow(row_data)

In [82]:
# Testing temp. range
def test_temp():
    for i in range(7):
        temperature = f'0.{i + 1}'
        response = generate_description(df['Diagnosis Category'][i], df['Diagnosis Sub Category'][i], df['Treatment Category'][i], df['Treatment Sub Category'][i], temperature)
        print(f'Row {i}, with temperature: {temperature}.\n', response, '\n')
        append_row(df['Visit ID'][i], response.content)
#test_temp()

In [92]:
import random
import time

temperatures = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

for i in range(7000, len(df)):
    df_descriptions = pd.read_csv('dataset_2.csv')
    if df['Visit ID'][i] not in df_descriptions['Visit ID'].unique():
        temperature = random.choice(temperatures)
        response = generate_description(df['Diagnosis Category'][i], df['Diagnosis Sub Category'][i], df['Treatment Category'][i], df['Treatment Sub Category'][i], temperature)
        append_row(df['Visit ID'][i], response.content)
        time.sleep(0.2)
    